# Exercise 1 - meta-nodes and offline pipelines 

For your first iteration, here is what you will build:
<img src="hello_world.png" alt='hello_world'>

Import usefull packages

In [1]:
from timeflux.helpers.testing import Looper, ReadData
from timeflux.core.branch import Branch
from utils.tutorial_utils import load_standalone_graph
from utils.idisplay import pygments_css, display_yaml

import logging
logging.basicConfig()
logger = logging.getLogger()
logger.setLevel(logging.DEBUG)


## Load the YAML graph 

In [2]:
graph_path = 'graphs/hello_world.yaml'
graph = load_standalone_graph(graph_path)
display_yaml(graph_path); 

graphs : 

 - nodes : 
 - id : random 
 module : timeflux.nodes.random 
 class : Random 
 params : 
 columns : 5 
 rows_min : 1 
 rows_max : 10 
 value_min : 0 
 value_max : 5 
 seed : 1 
 - id : add 
 module : timeflux_example.nodes.arithmetic 
 class : Add 
 params : 
 value : 1 
 - id : display_random 
 module : timeflux.nodes.debug 
 class : Display 
 - id : display_add 
 module : timeflux.nodes.debug 
 class : Display 
 edges : 
 - source : random 
 target : add 
 - source : random 
 target : display_random 
 - source : add 
 target : display_add

# Run from command line 
In a terminal, activate your timeflux environment and 
```
conda activate timeflux
```
and run the following 
```
timeflux graphs/hello_world.yaml -d
```

If you're not fealing comfortable with terminals, you can run the following cell and interupt the kernel to stop the process (Kernel --> Interrupt).

In [3]:
!timeflux graphs/hello_world.yaml -d

2019-12-05 15:07:38,071 INFO       timeflux     22746    MainProcess      Timeflux 0.3.3+121.g2fdae47.dirty
2019-12-05 15:07:38,082 DEBUG      manager      22746    MainProcess      Worker spawned with PID 22753
2019-12-05 15:07:38,438 DEBUG      debug        22753    Process-1        
                             0  1  2  3  4
2019-12-05 15:07:37.431592  5  3  4  0  1
2019-12-05 15:07:37.764925  3  5  0  0  1
2019-12-05 15:07:38.098258  4  5  4  1  2
2019-12-05 15:07:38,455 DEBUG      debug        22753    Process-1        
                             0  1  2  3  4
2019-12-05 15:07:37.431592  6  4  5  1  2
2019-12-05 15:07:37.764925  4  6  1  1  2
2019-12-05 15:07:38.098258  5  6  5  2  3
2019-12-05 15:07:39,438 DEBUG      debug        22753    Process-1        
                             0  1  2  3  4
2019-12-05 15:07:38.432195  4  5  2  4  3
2019-12-05 15:07:38.532195  4  2  4  5  2
2019-12-05 15:07:38.632195  4  1  1  0  5
2019-12-05 15:07:38.732195  1  1  5  1  1
2019-12-05 15:

# Use a meta-node to prototype offline

When one is prototyping a pipeline (developping custom nodes, writting a graph, .. ), it is very usefull to be able to loop manually, allowing him to use debug breakpoints, and check that each update of each node produces the result he expects.

Here, we use the concept of [branch](https://doc.timeflux.io/latest/extending/branches.html) to load a graph offline and mimick the scheduler *manually*, by `setting` the input ports and `getting` the output ports. 

In [4]:
from timeflux.core.registry import Registry
Registry.cycle_start = 0
Registry.rate = 1

branch = Branch(graph=graph)
branch.update()


DEBUG:timeflux.timeflux.nodes.debug.Display:
                             0  1  2  3  4
1969-12-31 23:59:59.000000  5  3  4  0  1
1969-12-31 23:59:59.333334  3  5  0  0  1
1969-12-31 23:59:59.666667  4  5  4  1  2
INFO:numexpr.utils:Note: NumExpr detected 12 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
INFO:numexpr.utils:NumExpr defaulting to 8 threads.
DEBUG:timeflux.timeflux.nodes.debug.Display:
                             0  1  2  3  4
1969-12-31 23:59:59.000000  6  4  5  1  2
1969-12-31 23:59:59.333334  4  6  1  1  2
1969-12-31 23:59:59.666667  5  6  5  2  3


In [5]:
branch.get_port('add', port_id='o').data

,0,1,2,3,4
1969-12-31 23:59:59.000000,6,4,5,1,2
1969-12-31 23:59:59.333334,4,6,1,1,2
1969-12-31 23:59:59.666667,5,6,5,2,3
